<a href="https://colab.research.google.com/github/ohashi-gnct/exp/blob/2022/A_google%E6%A4%9C%E7%B4%A2%E6%95%B0%E3%81%A8%E3%82%B3%E3%83%AD%E3%83%8A%E6%84%9F%E6%9F%93%E8%80%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import io
import json
import datetime as dt

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

Colabに最初から入っていないライブラリをUNIXコマンドの`pip`で導入する。

`japanize-matplotlib`は日本語でmatplotlibが使えるようにするライブラリである。

In [ ]:
!pip install japanize-matplotlib

In [ ]:
# matplotlibの日本語化
import japanize_matplotlib
%matplotlib inline
sns.set(font="IPAexGothic")

Googleドライブを接続し、CSVファイルを自分のドライブに保存する。

認証を求める画面が出る場合がある。

自身でJupyter notebook環境を作っている場合、ディレクトリは適当に読み替えること。

In [ ]:
# Googleドライブの接続
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_path = '/content/drive/MyDrive'

Mounted at /content/drive


プロットの見栄えがするように初期設定する。

In [ ]:
# matplotlibの初期設定
plt_config = {
    "figure.dpi": 150,
    "font.size": 14,
    "axes.xmargin": 0,
    "axes.ymargin": 0,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "mathtext.cal": "serif",
    "mathtext.rm": "serif",
    "mathtext.it": "serif:italic",
    "mathtext.bf": "serif.bold",
    "mathtext.fontset": "cm",
    "legend.framealpha": 1.0,
    "legend.edgecolor": "black",
    "legend.fancybox": False
}
plt.rcParams.update(plt_config)

# Google検索

特定の検索ワードについて、その検索数の推移を取得できる。

値は検索回数ではなく、検索回数の最大が100になるようスケールされた数である。

https://smart-hint.com/python/pytrends/

In [ ]:
!pip install pytrends

In [ ]:
from pytrends.request import TrendReq
pytrends = TrendReq(hl='ja-JP', tz=-540)

In [ ]:
kw_list = ['抗原検査'] # キーワードのリスト（5個まで）
# 開始、終了日を記述
pytrends.build_payload(kw_list=kw_list, 
                       timeframe='2021-07-29 2022-04-24',
                       geo='JP')
df_trends = pytrends.interest_over_time()
df_trends = df_trends.drop('isPartial',axis=1)
df_trends

In [ ]:
df_trends.plot()

データはCSVファイルとして保存する。

In [ ]:
df_trends.to_csv(os.path.join(drive_path, "trends.csv"))

In [ ]:
# 保存したCSVファイルを読み込みたい場合、以下のように記述する。
# df_trends = pd.read_csv(os.path.join(drive_path, "trends.csv"))
# df_trends["date"] = pd.to_datetime(df_trends["date"])
# df_trends = df_trends.set_index("date")

# コロナ感染者

NHKの新型コロナウイルス特設サイトからCSVをダウンロードする。

1日ごとの全国の感染者が記録されている。

https://www3.nhk.or.jp/news/special/coronavirus/data/

UNIXコマンド`curl`でGoogleドライブにファイルをダウンロードする。

In [ ]:
!curl -o /content/drive/MyDrive/covid19_cases.csv https://www3.nhk.or.jp/n-data/opendata/coronavirus/nhk_news_covid19_domestic_daily_data.csv

In [ ]:
df_covid = pd.read_csv(os.path.join(drive_path, "covid19_cases.csv"), 
                  names=("date", "cases"), usecols=[0, 1], skiprows=1)
df_covid["date"] = pd.to_datetime(df_covid["date"])
df_covid = df_covid.set_index("date")
df_covid

In [ ]:
df_covid.plot()

# 相関関係の分析

上のデータから任意の2データを選び、相関関係を分析する。

まずはDataFrameを結合する。

`index`つまり`date`が共通の列だけを結合した新たなDataFrameを作る。

In [ ]:
# 2つのdfでindexが共通の列だけを結合する
df_concat = pd.concat([df_trends, df_covid], axis=1, join="inner")
df_concat

今回は、第6波のデータだけ解析する。

配列の**スライス**を使って2021-12-01以降のデータを取り出す。

In [ ]:
df_concat = df_concat[-145:]
df_concat

それぞれの変数をプロットしてみる。

In [ ]:
df_concat.plot(subplots=True)

グラフを見ると、第6波が始まりだしたころに検索数が増え、

以降は感染者数が高止まりしているにも関わらず検索数は減少している。

他の（コロナウイルスに関係ない）ワードの検索傾向とも比較し、この理由を考察してもよい。

発表では相関分析や回帰分析以外の考察を説明してもよい。

matplotlibのラッパーである`seaborn`ライブラリを使って、

すべての組み合わせの2変数で散布図を作成できる。

In [ ]:
sns.pairplot(df_concat)

散布図を描くことで、2変数がどのように関係しているか直感的に確認できる。

この図をもとに考察してもよい。

これらの相関係数を求める。

3変数以上ある場合にも、相互に相関係数を求められる。

In [ ]:
corr = df_concat.corr()
sns.heatmap(corr, vmax=1, vmin=-1, center=0, 
            cmap="seismic", square=True, annot=True)

2変数の間にはある程度の相関があることがわかる。

# 単回帰分析

任意の1変数を目的変数(y軸)、別の任意の1変数を説明変数(x軸)として、

単回帰分析を行う。

まず、機械学習のためのライブラリscikit-learnから

単回帰分析のためのモジュールを読み込む。

In [ ]:
from sklearn import linear_model

In [ ]:
clf = linear_model.LinearRegression()
X = df_concat[["抗原検査"]]
Y = df_concat[["cases"]]
clf.fit(X, Y)

print("回帰係数: " + str(clf.coef_))
print("切片: " + str(clf.intercept_))
print("決定係数: " + str(clf.score(X, Y)))

決定係数は高くないため、このモデルでは目的変数を十分に説明できているわけではない。

回帰直線を散布図に重ねて描くと以下のようになる。

In [ ]:
sns.lmplot(x="抗原検査", y="cases", data=df_concat)

グラフからも、単回帰によって十分に目的変数を説明できないことがわかる。